### test_brisque_scores

Brisque (blind/referenceless image spatial quality evaluator) scores are a way of evaluating image quality.

This test uses the package at https://pypi.org/project/image-quality/   
but I did `conda install image-quality`

Here is a nice notebook that does all of the calculations from scratch: https://github.com/ocampor/notebooks/blob/master/notebooks/image/quality/brisque.ipynb

Mittal, A., Moorthy, A. K., & Bovik, A. C. (2012). No-Reference Image Quality Assessment in the Spatial Domain. IEEE Transactions on Image Processing, 21(12), 4695–4708. https://doi.org/10.1109/TIP.2012.2214050


In [1]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io, img_as_float
from skimage.transform import rescale

import glob
import imquality.brisque as brisque

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
%%time
impath = r'..\data\1581508801.c1.timex.jpg'
img = img_as_float(io.imread(impath, as_gray=True))
bs = brisque.score(img)
print('Brisque score:',bs)

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)
C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


Brisque score: 45.26572446406166
Wall time: 38.2 s


In [3]:
%%time
impath = r'..\data\1581508801.c1.timex.jpg'
img = rescale(img_as_float(io.imread(impath, as_gray=True)),.5,anti_aliasing=True)
bs = brisque.score(img)
print('Brisque score, scale factor 0.5:',bs)

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


Brisque score, scale factor 0.5: 27.297608834122997
Wall time: 9.85 s


In [4]:
%%time
impath = r'..\data\1581508801.c1.timex.jpg'
img = rescale(img_as_float(io.imread(impath, as_gray=True)),.25,anti_aliasing=True)
bs = brisque.score(img)
print('Brisque score, scale factor 0.25:',bs)

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


Brisque score, scale factor 0.25: 20.209473350875868
Wall time: 2.45 s


In [5]:
ofile = open('brisque_scores.csv','w')
for i, f in enumerate( glob.glob(r'C:\crs\proj\2019_CACO_CoastCam\random_obliques\*.jpg') ):
    img = img_as_float(io.imread(f, as_gray=True))
    bs = brisque.score(img)
    ts = '{}, {:.3f}\n'.format(f,bs)
    print(ts)
    ofile.write(ts)
#     plt.imshow(img,cmap='gray')
#     plt.title('BRISQUE score {:.2f}'.format(bs))
ofile.close()

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576261801.c1.timex.jpg, 51.351



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576261801.c2.timex.jpg, 52.376



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


...

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611091801.c2.timex.jpg, 63.371



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611241201.c1.timex.jpg, 42.412



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611241201.c2.timex.jpg, 47.699



In [6]:
ofile2 = open('brisque_scores_small.csv','w')
for i, f in enumerate( glob.glob(r'C:\crs\proj\2019_CACO_CoastCam\random_obliques\*.jpg') ):
    img = img_as_float(io.imread(f, as_gray=True))
    imgs = rescale(img,0.25,anti_aliasing=True)
    bs = brisque.score(imgs)
    ts = '{}, {:.3f}\n'.format(f,bs)
    print(ts)
    ofile2.write(ts)
#     plt.imshow(img,cmap='gray')
#     plt.title('BRISQUE score {:.2f}'.format(bs))
ofile2.close()

C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576261801.c1.timex.jpg, 21.911



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1576261801.c2.timex.jpg, 21.578



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


...

C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611091801.c2.timex.jpg, 31.262



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611241201.c1.timex.jpg, 10.492



C:\Users\csherwood\AppData\Local\Continuum\miniconda3\envs\CRS\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


C:\crs\proj\2019_CACO_CoastCam\random_obliques\1611241201.c2.timex.jpg, 21.641

